# 1. Iterator

Python iterator objects are required to support two methods while following the iterator protocol.

`__iter__` returns the iterator object itself. This is used in for and in statements.

`__next__` method returns the next value from the iterator. If there is no more items to return then it should raise **StopIteration** exception.

In [1]:
class CountDown(object):
    
    def __init__(self, start):
        self.counter = start + 1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.counter -= 1
        if self.counter <= 0:
            raise StopIteration
        return self.counter

In [2]:
c = CountDown(5)
for x in c:
    print(x)

5
4
3
2
1


In [47]:
c = CountDown(3)

In [4]:
next(c)


3

In [5]:
next(c)

2

In [6]:
next(c)

1

In [7]:
next(c)

StopIteration: 

In [8]:
from itertools import count
counter = count(start=10)
next(counter)

10

In [9]:
next(counter)

11

In [10]:
from itertools import cycle
colors = cycle(['red', 'white', 'blue'])
next(colors)

'red'

In [11]:
next(colors)

'white'

In [12]:
next(colors)

'blue'

In [13]:
next(colors)

'red'

In [14]:
from itertools import islice
colors = cycle(['red', 'white', 'blue'])  # infinite
limited = islice(colors, 0, 4)            # finite
for x in limited:                         # so safe to use for-loop on
    print(x)

red
white
blue
red


In [15]:
class fib(object):
    def __init__(self):
        self.prev = 0
        self.curr = 1
        
    def __iter__(self):
        return self
    
    def __next__(self):
        value = self.curr
        self.curr += self.prev
        self.prev = value
        return value
    
f = fib()
list(islice(f, 0, 10))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

# 2. Generator
They were introduced in Python 2.3. It is an easier way to create iterators using a keyword **yield** from a function.

A generator is a special kind of iterator—the elegant kind.

Let's be explicit:

- Any generator also is an iterator (not vice versa!);

- Any generator, therefore, is a factory that lazily produces values.

In [16]:
def infinite_generator(start=0):
    while True:
        yield start
        start += 1

for num in infinite_generator(4):
    print(num, end=' ')
    if num > 15:
        break

4 5 6 7 8 9 10 11 12 13 14 15 16 

In [17]:
def fib():
    prev, curr = 0, 1
    while True:
        yield curr
        prev, curr = curr, prev + curr
        
f = fib()
list(islice(f, 0, 11))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

### 2.1 Types of Generators
There are two types of generators in Python: generator **functions** and generator **expressions**. A generator function is any function in which the keyword `yield` appears in its body. We just saw an example of that. The appearance of the keyword yield is enough to make the function a generator function.

The other type of generators are the generator equivalent of a `list comprehension`. Its syntax is really elegant for a limited use case.

In [18]:
numbers = [1, 2, 3, 4, 5, 6]
# list comprehension
[x * x for x in numbers]

[1, 4, 9, 16, 25, 36]

In [20]:
# set comprehension
type({x * x for x in numbers})

set

In [21]:
# dict comprehension
{x: x * x for x in numbers}

{1: 1, 2: 4, 3: 9, 4: 16, 5: 25, 6: 36}

In [22]:
lazy_squares = (x * x for x in numbers)
lazy_squares

<generator object <genexpr> at 0x7fb1b577ed68>

In [23]:
next(lazy_squares)

1

In [24]:
list(lazy_squares)

[4, 9, 16, 25, 36]

# 3. Decorators
Decorator provide a very useful method to add functionality to existing functions and classes. Decorators are
functions that wrap other functions or classes.

In [25]:
def my_decorator(func):
    """ Testing doc """
    def wrapper(*args, **kwargs):
        print("Before call")
        result = func(*args, **kwargs)
        print("After call")
        return result
    return wrapper

In [48]:
@my_decorator
def add(a, b):
    "Our add function"
    return a + b

add = my_decorator(add)

In [49]:
add(1, 4)

Before call
Before call
After call
After call


5

In [28]:
my_decorator.__doc__

' Testing doc '

In [30]:
def mark(cls):
    cls.added_attr = 'I am decorated.'
    return cls

In [31]:
@mark
class A(object):
    pass

In [32]:
A.added_attr

'I am decorated.'

In [37]:
a = A()
b = A()
id(a.added_attr)

140401230598768

In [38]:
id(b.added_attr)

140401230598768

# 4. Singleton
Possibly the simplest design pattern is the `singleton`, which is a way to provide one and only one object of a particular type. To accomplish this, you must take control of object creation out of the hands of the programmer. One convenient way to do this is to delegate to a single instance of a private nested inner class:

In [40]:
class OnlyOne:
    class __OnlyOne:
        def __init__(self, arg):
            self.val = arg
        def __str__(self):
            return repr(self) + self.val
    instance = None
    def __init__(self, arg):
        if not OnlyOne.instance:
            OnlyOne.instance = OnlyOne.__OnlyOne(arg)
        else:
            OnlyOne.instance.val = arg
    def __getattr__(self, name):
        return getattr(self.instance, name)

    def __str__(self):
        # return repr(self) + self.instance.val
        return repr(self.instance) + self.instance.val

x = OnlyOne('sausage')
print(x)
y = OnlyOne('eggs')
print(y)
z = OnlyOne('spam')
print(z)
print(x)
print(y)

<__main__.OnlyOne.__OnlyOne object at 0x7fb1b5407470>sausage
<__main__.OnlyOne.__OnlyOne object at 0x7fb1b5407470>eggs
<__main__.OnlyOne.__OnlyOne object at 0x7fb1b5407470>spam
<__main__.OnlyOne.__OnlyOne object at 0x7fb1b5407470>spam
<__main__.OnlyOne.__OnlyOne object at 0x7fb1b5407470>spam


In [41]:
class Singleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]
    

    
class Mailer(metaclass=Singleton):
    pass

# 4. Factory Method
Factory Method is a creational design pattern used to create concrete implementations of a common interface.

It separates the process of creating an object from the code that depends on the interface of the object.

In [42]:
import json
import xml.etree.ElementTree as et


class Song(object):
    def __init__(self, song_id, title, artist):
        self.song_id = song_id
        self.title = title
        self.artist = artist


class SongSerializer(object):
    def serialize(self, song, format):
        serializer = self._get_serializer(format)
        return serializer(song)

    def _get_serializer(self, format):
        try:
            return {
                'JSON': self._serialize_to_json,
                'XML': self._serialize_to_xml
            }[format.upper()]
        except KeyError:
            raise KeyError(format)

    def _serialize_to_json(self, song):
        payload = {
            'id': song.song_id,
            'title': song.title,
            'artist': song.artist
        }
        return json.dumps(payload)

    def _serialize_to_xml(self, song):
        song_element = et.Element('song', attrib={'id': song.song_id})
        title = et.SubElement(song_element, 'title')
        title.text = song.title
        artist = et.SubElement(song_element, 'artist')
        artist.text = song.artist
        return et.tostring(song_element, encoding='unicode')

In [43]:
song = Song('1', 'Water of Love', 'Dire Straits')
serializer = SongSerializer()

serializer.serialize(song, 'JSON')

'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

In [44]:
serializer.serialize(song, 'xml')

'<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>'

In [45]:
serializer.serialize(song, 'XML')

'<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>'

In [46]:
serializer.serialize(song, 'YAML')

KeyError: 'YAML'